In [1]:
import pandas as pd
import numpy as np
import glob
from gplearn.genetic import SymbolicRegressor, SymbolicTransformer
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
import torch
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.utils import check_random_state
import time

In [2]:
mens = []
womens = []
labels = pd.read_csv('./gender_labels.csv')
for s in glob.glob('/neuro/notebooks/all_data_confounds_remove/*.csv'):
    person = int(s.split('/')[-1].split('_')[0])
    data = pd.read_csv(s)
    data = data.rolling(window=10).mean().dropna()
    if labels[labels['person']==person]['gender'].values[0]=='M':
        mens.append(data)
    else:
        womens.append(data)
mens = pd.concat(mens)
womens = pd.concat(womens)       

In [3]:
data = mens

In [4]:
# data = pd.read_csv('../notebooks/filter_with_confounds_dataset.csv')
region = 'x13'
X = data.drop([region], axis=1)
X = 10*X
feature_names = X.columns
feature_names = data.drop([region], axis=1).columns
y = data[region].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=0)

In [7]:
est_gp = SymbolicRegressor(population_size=500,
                           tournament_size=20,
                           generations=200, stopping_criteria=0.001,
                           const_range=(-3, 3),
                           p_crossover=0.7, p_subtree_mutation=0.12,
                           p_hoist_mutation=0.06, p_point_mutation=0.12,
                           p_point_replace=1,
                           init_depth = (10, 18),
                           function_set=('mul', 'sub', 'div', 'add', 'sin'),
#                            function_set=('mul', 'sub', 'add', 'sin'),
                           max_samples=0.9, 
                           verbose=1,
                           metric='mse',
                           parsimony_coefficient=0.00001, 
                           random_state=0, 
                           n_jobs=20)

est_gp.fit(X_train, y_train)
print('train:', r2_score(y_train, est_gp.predict(X_train)))
print('test:', r2_score(y_test, est_gp.predict(X_test)))
print('train:', MAE(y_train, est_gp.predict(X_train)))
print('test:', MAE(y_test, est_gp.predict(X_test)))
print('program:', est_gp._program)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0 12397.87     1.65116e+105    54409         0.248561         0.250694   1615.30m
   1  4614.18      2.18591e+62        3         0.248168         0.254233   1144.40m
   2  1250.27      1.47316e+08        3         0.247826         0.257316   1068.95m
   3   634.26      2.67949e+06        4         0.247413         0.261033    318.96m
   4   155.03      3.83946e+08        3         0.247617         0.259193    173.26m
   5     5.95      5.12237e+28        3         0.247473         0.260487     57.06m
   6     3.49      1.17432e+06        3         0.247447          0.26072     15.39m
   7     3.39      5.54371e+07        3         0.247406          0.26109     15.08m
   8     3.43          42245.4        3         0.247114         0.263717  

  94     4.33      7.77857e+08        3         0.247306         0.261987     10.03m
  95     3.84          24492.2        3         0.247374         0.261378      8.52m
  96     3.65       1.4345e+06        3         0.247351         0.261588      9.45m
  97     5.07       1.4148e+10        4         0.247253          0.26247      9.01m
  98     4.28      2.71572e+06        3         0.247333         0.261749      8.49m
  99     4.87      3.03841e+08        3         0.247291          0.26213      8.92m
 100     3.52          7172.83        3          0.24734         0.261685      7.97m
 101     3.41          10735.9        4         0.247429         0.260887      7.75m
 102     3.44          298.086        3         0.247188          0.26305      7.79m
 103     4.38      1.30251e+08        3         0.247236         0.262617      9.27m
 104     3.63       3.7324e+13        4         0.247576          0.25956      7.87m
 105     3.56      8.29786e+07        3         0.247152         

 191     6.15       8.7805e+15        3         0.247141         0.263477     41.01s
 192     3.76            34520        3         0.247429         0.260885     36.28s
 193     3.70      8.22635e+09        3         0.247304         0.262007     31.38s
 194     4.06      4.53631e+14        3         0.247425         0.260916     24.58s
 195     3.50      1.93722e+06        3         0.247291         0.262124     21.31s
 196     3.84      1.27533e+10        3         0.247492         0.260319     15.42s
 197     4.99       1.2085e+17        3         0.247555         0.259747     10.41s
 198     3.38          13544.2        3         0.247294           0.2621      4.80s
 199     3.31      3.36352e+07        3         0.246721          0.26726      0.00s
train: -5.679344274867049e-07
test: -5.3052601136638344e-06
train: 0.3875745943169553
test: 0.38908158095052214
program: sub(X21, X21)


In [6]:
model = Lasso(0.001)
model.fit(X_train, y_train)
print('train:', r2_score(y_train, model.predict(X_train)))
print('test:', r2_score(y_test, model.predict(X_test)))
print('train:', MAE(y_train, model.predict(X_train)))
print('test:', MAE(y_test, model.predict(X_test)))

train: 0.4700183706720499
test: 0.46690029553763723
train: 0.28775375337821785
test: 0.289285949988508


In [ ]:
est_gp = SymbolicRegressor(population_size=500,
                           tournament_size=20,
                           generations=100, stopping_criteria=0.01,
                           const_range=(-1, 1),
                           p_crossover=0.5, p_subtree_mutation=0.22,
                           p_hoist_mutation=0.06, p_point_mutation=0.22,
                           p_point_replace=1,
                           init_depth = (5, 10),
                           function_set=('mul', 'sub', 'div', 'add', 'sin'),
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.00001, 
                           random_state=0, 
                           n_jobs=20)

est_gp.fit(x, y)
print('program:', est_gp._program)